In [1]:
import melanoma as mel

%matplotlib inline
%load_ext autoreload
%autoreload -p 2

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")

import os
from string import Template

rootpath = '/hpcstor6/scratch01/s/sanghyuk.kim001'
# img_size = (224, 224) # height, width
# img_size = (150, 150) # height, width
utilInstance = mel.Util(rootpath)

2024-02-26 22:01:39.987544: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
SLURM_DIR = '/home/sanghyuk.kim001/MELANOMA/melanoma-detection-CNN/SLURMS'
# if os.path.isExist(SLURM_DIR) is not True:
#     os.makedirs(SLURM_DIR)

# #SBATCH -N 1 # Ensure that all cores are on one machine
# now = "$(date '+%Y%m%d_%H%M%S')"
# #SBATCH -n 2 # Number of cores
# #SBATCH --ntasks=1
# #SBATCH --ntasks-per-gpu=2
# #SBATCH --cpus-per-task=2
# #SBATCH --nodes=2
# #SBATCH --ntasks-per-core=1

# eval "$$(conda shell.bash hook)"
# eval "$$(shell-command)"


# #SBATCH --nodes 1 # Ensure that all cores are on one machine
# #SBATCH --ntasks=4
# #SBATCH --spread-job

# eval "$$(conda shell.bash hook)"
# conda activate clean_chimera_env

# #SBATCH --mem=128gb
# #SBATCH --array=1

# #SBATCH --ntasks-per-node=1
# #SBATCH --ntasks-per-core=1
# #SBATCH --cpus-per-task=1

# #SBATCH --gres=gpu:A100:1

# #SBATCH --nodes=1

#SBATCH -N 1 # Ensure that all cores are on one machine
#SBATCH -n 4 # Number of cores

# #SBATCH --mem=128gb

# export PYTHONUNBUFFERED=TRUE


SLURM_TEMPLATE = Template('''#!/bin/bash

#SBATCH --job-name=${DBname}_${self_aug}_${classifier}_${img_height}h_${img_width}w
#SBATCH -p haehn -q haehn_unlim
#SBATCH -w chimera12

#SBATCH -N 1 # Ensure that all cores are on one machine
#SBATCH -n 1 # Number of cores
#SBATCH --mem=128gb

#SBATCH --gres=gpu:1

#SBATCH -t 01-00:00

#SBATCH --output /home/sanghyuk.kim001/MELANOMA/melanoma-detection-CNN/SLURMS/LOGS/${classifier}/%x_%A_%a.out
#SBATCH --error /home/sanghyuk.kim001/MELANOMA/melanoma-detection-CNN/SLURMS/LOGS/${classifier}/%x_%A_%a.err
#SBATCH --array=1
##. /etc/profile,

eval "$$(conda shell.bash hook)"
conda activate clean_chimera_env

echo `date`


# For debugging purposes.
python --version
nvcc -V

# Print this sub-job's task ID
echo "My SLURM_ARRAY_TASK_ID: " $SLURM_ARRAY_TASK_ID

cd /home/sanghyuk.kim001/MELANOMA/melanoma-detection-CNN/

python train.py --DB $db --IMG_SIZE ${img_height} ${img_width} --CLASSIFIER $classifier --SELF_AUG ${self_aug} --JOB_INDEX $SLURM_ARRAY_TASK_ID

echo "Job ended!"

# end
exit 0;
''')

In [5]:
DBs = [db.name for db in mel.DatasetType]
Classifiers = [c.name for c in mel.NetworkType]

# import pickle
# datum = pickle.load(open('/hpcstor6/scratch01/s/sanghyuk.kim001/melanomaDB/customDB/DenseNet121/_7_point_criteria_150h_150w_binary.pkl', 'rb'))

combinedDBs = {
  # 1 DB
  'HAM10000': [mel.DatasetType.HAM10000.name],
  'ISIC2016': [mel.DatasetType.ISIC2016.name],
  'ISIC2017': [mel.DatasetType.ISIC2017.name],
  'ISIC2018': [mel.DatasetType.ISIC2018.name],
  'ISIC2019': [mel.DatasetType.ISIC2019.name],
  'ISIC2020': [mel.DatasetType.ISIC2020.name],
  # 'PH2': [mel.DatasetType.PH2.name],
  '_7_point_criteria': [mel.DatasetType._7_point_criteria.name],
  'PAD_UFES_20': [mel.DatasetType.PAD_UFES_20.name],
  'MEDNODE': [mel.DatasetType.MEDNODE.name],
  'KaggleMB': [mel.DatasetType.KaggleMB.name],

  # 2 DB
  'ISIC2016+ISIC2017': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name],
  'ISIC2016+ISIC2018': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2018.name],
  'ISIC2016+ISIC2019': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2019.name],
  'ISIC2016+ISIC2020': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2020.name],
  'ISIC2016+PH2': [mel.DatasetType.ISIC2016.name, mel.DatasetType.PH2.name],
  'ISIC2016+_7_point_criteria': [mel.DatasetType.ISIC2016.name, mel.DatasetType._7_point_criteria.name],
  'ISIC2016+PAD_UFES_20': [mel.DatasetType.ISIC2016.name, mel.DatasetType.PAD_UFES_20.name],
  'ISIC2016+MEDNODE': [mel.DatasetType.ISIC2016.name, mel.DatasetType.MEDNODE.name],
  'ISIC2016+KaggleMB': [mel.DatasetType.ISIC2016.name, mel.DatasetType.KaggleMB.name],
  # 3 DBs
  'ISIC2016+ISIC2017+ISIC2018': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name],
  'ISIC2016+ISIC2018+ISIC2019': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name],
  'ISIC2016+ISIC2019+ISIC2020': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name],
  'ISIC2016+ISIC2020+PH2': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name],
  'ISIC2016+PH2+_7_point_criteria': [mel.DatasetType.ISIC2016.name, mel.DatasetType.PH2.name, mel.DatasetType._7_point_criteria.name],
  'ISIC2016+_7_point_criteria+PAD_UFES_20': [mel.DatasetType.ISIC2016.name, mel.DatasetType._7_point_criteria.name, mel.DatasetType.PAD_UFES_20.name],
  'ISIC2016+PAD_UFES_20+MEDNODE': [mel.DatasetType.ISIC2016.name, mel.DatasetType.PAD_UFES_20.name, mel.DatasetType.MEDNODE.name],
  'ISIC2016+MEDNODE+KaggleMB': [mel.DatasetType.ISIC2016.name, mel.DatasetType.MEDNODE.name, mel.DatasetType.KaggleMB.name],

  # 4 DBs
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name],
  'ISIC2016+ISIC2018+ISIC2019+ISIC2020': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name],
  'ISIC2016+ISIC2020+PH2+_7_point_criteria': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name, mel.DatasetType._7_point_criteria.name],
  'ISIC2016+PH2+_7_point_criteria+PAD_UFES_20': [mel.DatasetType.ISIC2016.name, mel.DatasetType.PH2.name, mel.DatasetType._7_point_criteria.name, mel.DatasetType.PAD_UFES_20.name],
  'ISIC2016+_7_point_criteria+PAD_UFES_20+MEDNODE': [mel.DatasetType.ISIC2016.name, mel.DatasetType._7_point_criteria.name, mel.DatasetType.PAD_UFES_20.name, mel.DatasetType.MEDNODE.name],
  'ISIC2016+PAD_UFES_20+MEDNODE+KaggleMB': [mel.DatasetType.ISIC2016.name, mel.DatasetType.PAD_UFES_20.name, mel.DatasetType.MEDNODE.name, mel.DatasetType.KaggleMB.name],

  # 5 DBs
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2020+PH2': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name],
  'ISIC2016+ISIC2017+ISIC2018+PH2+_7_point_criteria': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.PH2.name, mel.DatasetType._7_point_criteria.name],
  'ISIC2016+ISIC2017+ISIC2018+_7_point_criteria+PAD_UFES_20': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType._7_point_criteria.name, mel.DatasetType.PAD_UFES_20.name],
  'ISIC2016+ISIC2017+ISIC2018+PAD_UFES_20+MEDNODE': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.PAD_UFES_20.name, mel.DatasetType.MEDNODE.name],
  'ISIC2016+ISIC2017+ISIC2018+MEDNODE+KaggleMB': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.MEDNODE.name, mel.DatasetType.KaggleMB.name],

  # 6 DBs
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+PH2': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+PH2+_7_point_criteria': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.PH2.name, mel.DatasetType._7_point_criteria.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+_7_point_criteria+PAD_UFES_20': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType._7_point_criteria.name, mel.DatasetType.PAD_UFES_20.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+PAD_UFES_20+MEDNODE': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.PAD_UFES_20.name, mel.DatasetType.MEDNODE.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+MEDNODE+KaggleMB': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.MEDNODE.name, mel.DatasetType.KaggleMB.name],

  # 7 DBs
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+PH2+_7_point_criteria': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name, mel.DatasetType._7_point_criteria.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+_7_point_criteria+PAD_UFES_20': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType._7_point_criteria.name, mel.DatasetType.PAD_UFES_20.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+PAD_UFES_20+MEDNODE': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PAD_UFES_20.name, mel.DatasetType.MEDNODE.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+MEDNODE+KaggleMB': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.MEDNODE.name, mel.DatasetType.KaggleMB.name],

  # 8 DBs
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+PH2+_7_point_criteria+PAD_UFES_20': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name, mel.DatasetType._7_point_criteria.name, mel.DatasetType.PAD_UFES_20.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+PH2+PAD_UFES_20+MEDNODE': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name, mel.DatasetType.PAD_UFES_20.name, mel.DatasetType.MEDNODE.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+PH2+MEDNODE+KaggleMB': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name, mel.DatasetType.MEDNODE.name, mel.DatasetType.KaggleMB.name],

  # 9 DBs
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+PH2+_7_point_criteria+PAD_UFES_20+MEDNODE': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name, mel.DatasetType._7_point_criteria.name, mel.DatasetType.PAD_UFES_20.name, mel.DatasetType.MEDNODE.name],
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+PH2+_7_point_criteria+MEDNODE+KaggleMB': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name, mel.DatasetType._7_point_criteria.name, mel.DatasetType.MEDNODE.name, mel.DatasetType.KaggleMB.name],

  # 10 DBs
  'ISIC2016+ISIC2017+ISIC2018+ISIC2019+ISIC2020+PH2+_7_point_criteria+PAD_UFES_20+MEDNODE+KaggleMB': [mel.DatasetType.ISIC2016.name, mel.DatasetType.ISIC2017.name, mel.DatasetType.ISIC2018.name, mel.DatasetType.ISIC2019.name, mel.DatasetType.ISIC2020.name, mel.DatasetType.PH2.name, mel.DatasetType._7_point_criteria.name, mel.DatasetType.PAD_UFES_20.name, mel.DatasetType.MEDNODE.name, mel.DatasetType.KaggleMB.name],

}


img_size = (150, 150)


# Single DB
# for d in DBs:
#   for c in Classifiers:
#     new_slurm = SLURM_TEMPLATE.substitute(db=[d], memory=32000, img_size=img_size, classifier=c, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
#     slurm_file = os.path.join(SLURM_DIR, d+'_'+c+'.sh')
#     with open(slurm_file, 'w') as f:
#       f.write(new_slurm)

# if len(combinedDBs) == 1:
#   DBname = DB
# elif len(combinedDBs) > 1:
#   DBname = '+'.join(DB)
import os

for c in Classifiers:
  
  for idx, comb_d in enumerate(combinedDBs):
    
    num_of_comb = len(combinedDBs[comb_d])

    DBname = '+'.join(combinedDBs[comb_d])
    noaug_new_slurm = SLURM_TEMPLATE.substitute(db=' '.join(combinedDBs[comb_d]), DBname=DBname, self_aug=0, img_height=img_size[0], img_width=img_size[1], memory=8000, classifier=c, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
    aug_new_slurm = SLURM_TEMPLATE.substitute(db=' '.join(combinedDBs[comb_d]), DBname=DBname, self_aug=1, img_height=img_size[0], img_width=img_size[1], memory=8000, classifier=c, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
    
    combinedDB_name = '+'.join(combinedDBs[comb_d])

    # division = idx % 2
    

    # folder = SLURM_DIR+'/'+c + '/' + str(num_of_comb)
    
    noaug_folder = SLURM_DIR + '/' + 'noaug' + '/' + c
    aug_folder = SLURM_DIR + '/' + 'aug' + '/' + c

    if os.path.exists(noaug_folder) is not True:
      os.makedirs(noaug_folder)

    if os.path.exists(aug_folder) is not True:
      os.makedirs(aug_folder)



    noaug_slurm_file = os.path.join(noaug_folder, combinedDB_name+'_'+'noaug'+'_'+c+'.sbatch')
    aug_slurm_file = os.path.join(aug_folder, combinedDB_name+'_'+'aug'+'_'+c+'.sbatch')
    with open(noaug_slurm_file, 'w') as f:
      f.write(noaug_new_slurm)
    with open(aug_slurm_file, 'w') as f:
      f.write(aug_new_slurm)


# # Combined DBs
# for idx, comb_d in combinedDBs:
#   division = idx % 2

#   folder = SLURM_DIR+'/'+c + '/' + str(division)
#   if os.path.exists(folder) is not True:
#     os.makedirs(folder)
#   for c in enumerate(Classifiers):
    
    

#     DBname = '+'.join(combinedDBs[comb_d])
#     new_slurm = SLURM_TEMPLATE.substitute(db=' '.join(combinedDBs[comb_d]), DBname=DBname, img_height=img_size[0], img_width=img_size[1], memory=8000, classifier=c, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
#     combinedDB_name = '+'.join(combinedDBs[comb_d])

    

#     slurm_file = os.path.join(folder, combinedDB_name+'_'+c+'.sbatch')
#     with open(slurm_file, 'w') as f:
#       f.write(new_slurm)

# slurm_file = os.path.join(SLURM_DIR, DBtemp[0]+'_'+Classifiers_temp[0]+'.sh')
# new_slurm = SLURM_TEMPLATE.substitute(db=d, memory=32000, classifier=c, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
# with open(os.path.join(SLURM_DIR, db+'_'+c+'.sbatch'), 'w') as f:
#   f.write(new_slurm)



In [31]:

# import subprocess
# subprocess.call(['sh', f'./{slurm_file}'])

import itertools
import glob

import os
batches = list(itertools.chain.from_iterable([glob.glob(f'{SLURM_DIR}/*.sh', recursive=True)]))
for b in batches:
    os.system(f'sbatch {b}')


Submitted batch job 310786
Submitted batch job 310787
Submitted batch job 310788
Submitted batch job 310789
Submitted batch job 310790
Submitted batch job 310791
Submitted batch job 310792
Submitted batch job 310793
Submitted batch job 310794
Submitted batch job 310795
Submitted batch job 310796
Submitted batch job 310797
Submitted batch job 310798
Submitted batch job 310799
Submitted batch job 310800
Submitted batch job 310801
Submitted batch job 310802
Submitted batch job 310803
Submitted batch job 310804
Submitted batch job 310805
Submitted batch job 310806
Submitted batch job 310807
Submitted batch job 310808
Submitted batch job 310809
Submitted batch job 310810
Submitted batch job 310811
Submitted batch job 310812
Submitted batch job 310813
Submitted batch job 310814
Submitted batch job 310815
Submitted batch job 310816
Submitted batch job 310817
Submitted batch job 310818
Submitted batch job 310819
Submitted batch job 310820
Submitted batch job 310821
Submitted batch job 310822
S

In [ ]:
for f in *.sbatch; do sbatch $f; done

In [2]:
%run train.py --DB PAD_UFES_20 --IMG_SIZE 150 150 --CLASSIFIER Xception --JOB_INDEX 1

DB: ['PAD_UFES_20']
IMG_SIZE: [150, 150]
CLASSIFIER: Xception
JOB_INDEX: 1
Combining...
Combining 1 db out of 1 dbs
Stacking training images
Stacking training labels
Stacking validation images
Stacking validation labels
Combining...
Combining 1 db out of 1 dbs
Stacking training images
Stacking training labels
Stacking validation images
Stacking validation labels


2024-02-26 14:30:44.935504: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-02-26 14:30:45.063865: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2024-02-26 14:30:45.063911: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-02-26 14:30:45.095497: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-02-26 14:30:45.095541: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2024-02-26 14:30:45.109626: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.10
2

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 2048)              20861480  
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 256)               1

2024-02-26 14:30:53.279978: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-02-26 14:30:53.299162: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2245720000 Hz
2024-02-26 14:30:56.263115: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2024-02-26 14:30:57.106592: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2024-02-26 14:30:58.146898: W tensorflow/stream_executor/gpu/asm_compiler.cc:191] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2024-02-26 14:30:58.146923: W tensorflow/stream_executor/gpu/asm_compiler.cc:194] Used ptxas at ptxas
2024-02-26 14:30:58.147449: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modif

Epoch: 1 loss: 0.6564 accuracy: 0.6805 val_loss: 0.6124 val_accuracy: 0.9804
Epoch: 2 loss: 0.5745 accuracy: 0.9452 val_loss: 0.5276 val_accuracy: 0.9804
Epoch: 3 loss: 0.4864 accuracy: 0.9767 val_loss: 0.4351 val_accuracy: 0.9804
Epoch: 4 loss: 0.3988 accuracy: 0.9767 val_loss: 0.3528 val_accuracy: 0.9804
Epoch: 5 loss: 0.3214 accuracy: 0.9767 val_loss: 0.2854 val_accuracy: 0.9804
Epoch: 6 loss: 0.2604 accuracy: 0.9762 val_loss: 0.2295 val_accuracy: 0.9804
Epoch: 7 loss: 0.2151 accuracy: 0.9762 val_loss: 0.1887 val_accuracy: 0.9804
Epoch: 8 loss: 0.1817 accuracy: 0.9767 val_loss: 0.1610 val_accuracy: 0.9804
Epoch: 9 loss: 0.1595 accuracy: 0.9767 val_loss: 0.1404 val_accuracy: 0.9804
Epoch: 10 loss: 0.1448 accuracy: 0.9762 val_loss: 0.1292 val_accuracy: 0.9804
Epoch: 11 loss: 0.1338 accuracy: 0.9767 val_loss: 0.1232 val_accuracy: 0.9804
Epoch: 12 loss: 0.1266 accuracy: 0.9767 val_loss: 0.1158 val_accuracy: 0.9804
Epoch: 13 loss: 0.1200 accuracy: 0.9773 val_loss: 0.1093 val_accuracy: 0.

/home/sanghyuk.kim001/anaconda3/envs/clean_chimera_env/lib/python3.9/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


ValueError: Unable to create dataset (name already exists)

In [1]:
%run test.py

2024-02-26 13:29:40.195526: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Testing on 7-point-criteria DB
Evaluating ISIC2016+PH2+_7_point_criteria+PAD_UFES_20_noaug_VGG16_150h_150w_1 model on ISIC2018...



2024-02-26 13:29:44.433882: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-02-26 13:29:44.570543: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2024-02-26 13:29:44.570584: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-02-26 13:29:44.574173: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-02-26 13:29:44.574209: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2024-02-26 13:29:44.575292: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.10
2

11/13 [========================>.....] - ETA: 0s - loss: 0.1288 - accuracy: 0.9886

2024-02-26 13:29:49.236047: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


13/13 [==============================] - 4s 89ms/step - loss: 0.1335 - accuracy: 0.9903
Train loss = 0.1335097700357437  ;  Train accuracy = 99.03%

7/7 [==============================] - 1s 108ms/step - loss: 0.6192 - accuracy: 0.7537
Validation loss = 0.6191590428352356  ;  Validation accuracy = 75.37%

13/13 [==============================] - 0s 14ms/step - loss: 0.4636 - accuracy: 0.8076
Test loss = 0.4635915160179138  ;  Test accuracy = 80.76%
Computing predictions for ISIC2016+PH2+_7_point_criteria+PAD_UFES_20_noaug_VGG16_150h_150w_1 on ISIC2018...
Model report for ISIC2016+PH2+_7_point_criteria+PAD_UFES_20_noaug_VGG16_150h_150w_1 model ->


Train Report :
 {'Benign': {'precision': 0.9938080495356038, 'recall': 0.9938080495356038, 'f1-score': 0.9938080495356038, 'support': 323}, 'Malignant': {'precision': 0.9777777777777777, 'recall': 0.9777777777777777, 'f1-score': 0.9777777777777777, 'support': 90}, 'accuracy': 0.9903147699757869, 'macro avg': {'precision': 0.9857929136566908, 